In [44]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [45]:
# Load the historical weather data
historical_data = pd.read_csv('./Data/historical_weather_cleaned_filtered.csv')

# Load the submission key
submission_key = pd.read_csv('./Data/submission_key.csv')

In [46]:
# Pivot the historical data to get cities as columns and dates as rows
avg_temp_df = historical_data.pivot(index='date', columns='city_id', values='avg_temp_c')
min_temp_df = historical_data.pivot(index='date', columns='city_id', values='min_temp_c')
max_temp_df = historical_data.pivot(index='date', columns='city_id', values='max_temp_c')

# Convert the index to datetime
avg_temp_df.index = pd.to_datetime(avg_temp_df.index)

In [47]:
# Train an ARIMA model for each city using the historical data up to the end of 2018
predictions = {}

for city in avg_temp_df.columns:
    # Train the ARIMA model on data up to the end of 2018
    train_data = avg_temp_df[avg_temp_df.index < '2019-01-01'][city]
    
    # Fit the model
    model = ARIMA(train_data, order=(5, 1, 0))  # The order (5, 1, 0) can be adjusted based on performance
    model_fit = model.fit()
    
    # Forecast the average temperature for the first week of 2019
    forecast = model_fit.forecast(steps=7)
    
    # Store the forecast in the predictions dictionary
    predictions[city] = forecast.values

c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is ava

In [48]:
# Create a DataFrame for the submission
submission = submission_key.copy()
submission['avg_temp_c'] = submission.apply(lambda row: predictions[row['city_id']][(pd.to_datetime(row['date']) - pd.to_datetime('2019-01-01')).days], axis=1)

submission.head()

,submission_ID,city_id,date,avg_temp_c
0,1,C001,2019-01-01,11.548374
1,2,C001,2019-01-02,11.087903
2,3,C001,2019-01-03,10.936680
3,4,C001,2019-01-04,10.840372
4,5,C001,2019-01-05,10.863584


In [49]:
# Use only submission_ID,avg_temp_c columns
submission = submission[['submission_ID', 'avg_temp_c']]

# Save the submission to a CSV file
submission.to_csv('./Data/submission_ARIMA.csv', index=False)

print("Predicted temperatures for submission:")
print(submission)

Predicted temperatures for submission:
     submission_ID  avg_temp_c
0                1   11.548374
1                2   11.087903
2                3   10.936680
3                4   10.840372
4                5   10.863584
..             ...         ...
695            696   21.307010
696            697   21.402088
697            698   21.227765
698            699   21.124234
699            700   21.132516

[700 rows x 2 columns]


In [50]:
# Print the average temperature data for each city for each day for last day of 2018
print("Last week of 2018:")
avg_temp_df[avg_temp_df.index == '2018-12-31']

Last week of 2018:


city_id,C001,C002,C003,C004,C005,C007,C008,C009,C010,C011,...,C103,C104,C105,C106,C107,C108,C109,C110,C111,C112
date,,,,,,,,,,,,,,,,,,,,,
2018-12-31,11.8,14.0,27.1,1.9,26.0,20.2,5.2,18.4,16.8,2.0,...,2.3,28.5,6.4,21.7,12.7,23.0,1.4,27.9,28.1,20.2


In [51]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [52]:
# Dictionary to store predictions
predictions = {}

# Train a SARIMA model for each city using the historical data up to the end of 2018
for city in avg_temp_df.columns:
    # Filter training data up to the end of 2018
    train_data = avg_temp_df[avg_temp_df.index < '2019-01-01'][city]
    
    # Fit the SARIMA model
    model = SARIMAX(train_data, order=(5, 1, 0), seasonal_order=(1, 0, 1, 7))  # SARIMA parameters (p, d, q, P, D, Q, S)
    model_fit = model.fit(disp=False)
    
    # Forecast the average temperature for the first week of 2019
    forecast = model_fit.forecast(steps=7)
    
    # Store the forecast in the predictions dictionary
    predictions[city] = forecast.values

c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the 

In [53]:
# Create a DataFrame for the submission
submission = submission_key.copy()

# Function to retrieve predicted temperature for each row in submission_key
def get_predicted_temperature(row):
    city_id = row['city_id']
    date = pd.to_datetime(row['date'])
    days_since_start = (date - pd.to_datetime('2019-01-01')).days
    return predictions[city_id][days_since_start]

In [54]:
# Apply the function to create the 'avg_temp_c' column in submission
submission['avg_temp_c'] = submission.apply(get_predicted_temperature, axis=1)

# Use only submission_ID,avg_temp_c columns
submission = submission[['submission_ID', 'avg_temp_c']]

# Save the submission to a CSV file
submission.to_csv('./Data/submission_SARIMA.csv', index=False)

print("Predicted temperatures for submission:")
print(submission)

Predicted temperatures for submission:
     submission_ID  avg_temp_c
0                1   11.427853
1                2   10.849132
2                3   10.560276
3                4   10.231235
4                5   10.344537
..             ...         ...
695            696   21.158937
696            697   21.306203
697            698   21.111720
698            699   21.044020
699            700   21.139931

[700 rows x 2 columns]


In [55]:
from xgboost import XGBRegressor

In [56]:
# Function to prepare data for XGBoost
def prepare_xgb_data(city_data, n_steps=7):
    X, y = [], []
    for i in range(n_steps, len(city_data)):
        X.append(city_data[i-n_steps:i])
        y.append(city_data[i])
    return np.array(X), np.array(y)

# Function to train XGBoost model
def train_xgb_model(train_data, test_data):
    predictions = {}
    rmse_list = []
    for city in avg_temp_df.columns:
        train_series = train_data[city].dropna()
        test_series = test_data[city].dropna()

        X_train, y_train = prepare_xgb_data(train_series.values)
        X_test, y_test = prepare_xgb_data(np.concatenate([train_series.values[-7:], test_series.values]))

        # Initialize and fit the model
        model = XGBRegressor(objective='reg:squarederror')
        model.fit(X_train, y_train)

        # Forecast the average temperature for the test set period (2018-12-25 to 2018-12-31)
        forecast = model.predict(X_test)

        # Store the forecast in the predictions dictionary
        predictions[city] = forecast

        # Calculate RMSE for the city
        rmse = np.sqrt(mean_squared_error(y_test, forecast))
        rmse_list.append(rmse)
        print(f"RMSE for {city}: {rmse}")

    # Calculate the average RMSE across all cities
    average_rmse = np.mean(rmse_list)
    print(f"Average RMSE across all cities: {average_rmse}")

    return predictions

In [57]:
# Split the data into training and test sets
train_data = avg_temp_df[:'2018-12-24']
test_data = avg_temp_df['2018-12-25':'2018-12-31']

In [58]:
from sklearn.metrics import mean_squared_error

In [59]:
# Train XGBoost models and get predictions
predictions_xgb = train_xgb_model(train_data, test_data)

RMSE for C001: 1.2543542621941288
RMSE for C002: 1.3930397189111925
RMSE for C003: 0.7143296746041221
RMSE for C004: 3.2131002522854133
RMSE for C005: 1.919338312345123
RMSE for C007: 2.98233723376469
RMSE for C008: 2.9562611738599855
RMSE for C009: 1.8972392559863018
RMSE for C010: 1.7733905369980023
RMSE for C011: 3.044564093819313
RMSE for C012: 3.384837011227436
RMSE for C013: 2.4274943235905386
RMSE for C014: 3.2850794062776227
RMSE for C015: 0.8140046448091395
RMSE for C016: 0.7978850486322383
RMSE for C017: 1.1533000540588514
RMSE for C018: 2.4848195207156167
RMSE for C020: 0.8714608413641772
RMSE for C022: 2.249101643225109
RMSE for C023: 2.5083412534976706
RMSE for C024: 2.402268332527466
RMSE for C025: 5.010264662984513
RMSE for C027: 4.836477769610275
RMSE for C028: 0.3517883986053112
RMSE for C029: 1.109328477481987
RMSE for C030: 0.652635785742896
RMSE for C031: 3.0567490617193753
RMSE for C033: 2.784621578679616
RMSE for C034: 3.5998881016996567
RMSE for C035: 0.628250274

In [60]:
# Prepare output DataFrame for submission
submission = submission_key.copy()

In [61]:
# Function to retrieve predicted temperature for each row in submission_key
def get_predicted_temperature(row):
    city_id = row['city_id']
    date_index = (pd.to_datetime(row['date']) - pd.to_datetime('2019-01-01')).days % 7
    return predictions_xgb[city_id][date_index]

# Apply the function to create the 'avg_temp_c' column in submission
submission['avg_temp_c'] = submission.apply(get_predicted_temperature, axis=1)


In [62]:
# Use only submission_ID, avg_temp_c columns
submission = submission[['submission_ID', 'avg_temp_c']]

# Save the submission to a CSV file
submission.to_csv('./Data/submission_XGBoost.csv', index=False)

In [63]:
# Split data into training and test sets
train_data = avg_temp_df[:'2018-12-24']

# List to store predictions for the first week of 2019
predictions = pd.DataFrame(index=pd.date_range(start='2019-01-01', periods=7, freq='D'), columns=avg_temp_df.columns)

# Train a SARIMA model for each city using min and max temperatures and predict average temperature for the first week of 2019
for city in avg_temp_df.columns:
    # Train the SARIMA model using min temperature
    model_min = SARIMAX(min_temp_df[city], order=(5, 1, 0), seasonal_order=(1, 0, 1, 7))
    model_fit_min = model_min.fit(disp=False)
    forecast_min = model_fit_min.get_forecast(steps=7)
    
    # Train the SARIMA model using max temperature
    model_max = SARIMAX(max_temp_df[city], order=(5, 1, 0), seasonal_order=(1, 0, 1, 7))
    model_fit_max = model_max.fit(disp=False)
    forecast_max = model_fit_max.get_forecast(steps=7)
    
    # Calculate average temperature forecast based on min and max temperature forecasts
    forecast_avg = (forecast_min.predicted_mean + forecast_max.predicted_mean) / 2
    
    # Store the forecast in the predictions DataFrame
    predictions[city] = forecast_avg.values

# Print predictions for the first week of 2019
print("Predictions for the first week of 2019 using SARIMA (Average Temperature):")
print(predictions)

c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the 

Predictions for the first week of 2019 using SARIMA (Average Temperature):
city_id          C001       C002       C003      C004       C005       C007  \
2019-01-01  11.685527  15.643451  27.435756  0.612626  28.318946  20.342824   
2019-01-02  11.416550  15.495642  27.523073 -1.026284  26.935258  20.201860   
2019-01-03  11.362590  15.520900  27.453332 -1.602996  26.403727  19.990239   
2019-01-04  11.143934  15.396596  27.575530 -1.726018  26.175897  19.956958   
2019-01-05  11.245080  15.511979  27.474209 -0.946475  26.493697  19.825656   
2019-01-06  11.249998  15.555268  27.471447 -0.376989  26.585791  19.997227   
2019-01-07  11.233562  15.554856  27.469496 -0.907288  26.402497  20.074671   

city_id         C008       C009       C010      C011  ...      C103  \
2019-01-01  5.055447  18.341157  18.962974 -0.168854  ...  4.216698   
2019-01-02  4.935053  18.414485  19.279047  0.079195  ...  4.528667   
2019-01-03  4.426670  18.306296  19.516045  0.058705  ...  4.545491   
2019-01-

c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [64]:
# Create a DataFrame for the submission
submission = submission_key.copy()

In [65]:
def get_predicted_temperature(row):
    city_id = row['city_id']
    date = pd.to_datetime(row['date'])
    days_since_start = (date - pd.to_datetime('2019-01-01')).days
    return predictions[city_id][days_since_start]

# Apply the function to create the 'avg_temp_c' column in submission
submission['avg_temp_c'] = submission.apply(get_predicted_temperature, axis=1)

C:\Users\kauad\AppData\Local\Temp\ipykernel_14596\2279469520.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return predictions[city_id][days_since_start]


In [66]:
# Use only submission_ID and avg_temp_c columns
submission = submission[['submission_ID', 'avg_temp_c']]

# Save the submission to a CSV file
submission.to_csv('./Data/submission_SARIMA_with_MinMax.csv', index=False)

In [67]:
print("Predicted temperatures for submission:")
print(submission)

Predicted temperatures for submission:
     submission_ID  avg_temp_c
0                1   11.685527
1                2   11.416550
2                3   11.362590
3                4   11.143934
4                5   11.245080
..             ...         ...
695            696   21.447137
696            697   21.597475
697            698   21.645013
698            699   21.475731
699            700   21.502210

[700 rows x 2 columns]


In [68]:
# compute the rmse for average temperature of './Data/submission_SARIMA_with_MinMax.csv' using base './Data/submission_SARIMA.csv'
submission_base = pd.read_csv('./Data/submission_SARIMA.csv')
submission_with_minmax = pd.read_csv('./Data/submission_SARIMA_with_MinMax.csv')

# Merge the two submissions on 'submission_ID'
merged_submissions = submission_base.merge(submission_with_minmax, on='submission_ID', suffixes=('_base', '_with_minmax'))

# Calculate the RMSE between the two sets of predictions
rmse = np.sqrt(mean_squared_error(merged_submissions['avg_temp_c_base'], merged_submissions['avg_temp_c_with_minmax']))
print(f"RMSE between base and MinMax predictions: {rmse}")

RMSE between base and MinMax predictions: 0.8226964222501816


In [69]:
# Split data into training and test sets
train_data = avg_temp_df[:'2017-12-24']

# List to store predictions for the first week of 2019
predictions = pd.DataFrame(index=pd.date_range(start='2018-01-01', periods=7, freq='D'), columns=avg_temp_df.columns)

# Train a SARIMA model for each city using min and max temperatures and predict average temperature for the first week of 2019
for city in avg_temp_df.columns:
    # Train the SARIMA model using min temperature
    model_min = SARIMAX(min_temp_df[city], order=(5, 1, 0), seasonal_order=(1, 0, 1, 7))
    model_fit_min = model_min.fit(disp=False)
    forecast_min = model_fit_min.get_forecast(steps=7)
    
    # Train the SARIMA model using max temperature
    model_max = SARIMAX(max_temp_df[city], order=(5, 1, 0), seasonal_order=(1, 0, 1, 7))
    model_fit_max = model_max.fit(disp=False)
    forecast_max = model_fit_max.get_forecast(steps=7)
    
    # Calculate average temperature forecast based on min and max temperature forecasts
    forecast_avg = (forecast_min.predicted_mean + forecast_max.predicted_mean) / 2
    
    # Store the forecast in the predictions DataFrame
    predictions[city] = forecast_avg.values

# Print predictions for the first week of 2019
print("Predictions for the first week of 2018 using SARIMA (Average Temperature):")
print(predictions)

c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the 

KeyboardInterrupt: 

In [71]:
# Load the historical weather data
historical_data = pd.read_csv('./Data/historical_weather_cleaned_filtered.csv')

# Pivot the historical data to get cities as columns and dates as rows
avg_temp_df = historical_data.pivot(index='date', columns='city_id', values='avg_temp_c')
min_temp_df = historical_data.pivot(index='date', columns='city_id', values='min_temp_c')
max_temp_df = historical_data.pivot(index='date', columns='city_id', values='max_temp_c')

# Convert the index to datetime
avg_temp_df.index = pd.to_datetime(avg_temp_df.index)
min_temp_df.index = pd.to_datetime(min_temp_df.index)
max_temp_df.index = pd.to_datetime(max_temp_df.index)

# Split the data into training and test sets
train_data = avg_temp_df[:'2017-12-31']
test_data = avg_temp_df['2018-01-01':'2018-01-07']

# List to store predictions for the first week of 2018
predictions = pd.DataFrame(index=pd.date_range(start='2018-01-01', periods=7, freq='D'), columns=avg_temp_df.columns)

# Train a SARIMA model for each city using min and max temperatures and predict average temperature for the first week of 2018
for city in avg_temp_df.columns:
    # Train the SARIMA model using min temperature
    model_min = SARIMAX(min_temp_df[city][:train_data.index[-1]], order=(5, 1, 0), seasonal_order=(1, 0, 1, 7))
    model_fit_min = model_min.fit(disp=False)
    forecast_min = model_fit_min.get_forecast(steps=7)
    
    # Train the SARIMA model using max temperature
    model_max = SARIMAX(max_temp_df[city][:train_data.index[-1]], order=(5, 1, 0), seasonal_order=(1, 0, 1, 7))
    model_fit_max = model_max.fit(disp=False)
    forecast_max = model_fit_max.get_forecast(steps=7)
    
    # Calculate average temperature forecast based on min and max temperature forecasts
    forecast_avg = (forecast_min.predicted_mean + forecast_max.predicted_mean) / 2
    
    # Store the forecast in the predictions DataFrame
    predictions[city] = forecast_avg.values

# Print predictions for the first week of 2018
print("Predictions for the first week of 2018 using SARIMA (Average Temperature):")
print(predictions)

# Calculate RMSE for each city and store in a list
rmse_list = []
for city in avg_temp_df.columns:
    actual = test_data[city]
    predicted = predictions[city]
    
    # Check for valid indices
    valid_indices = ~np.isnan(actual) & ~np.isnan(predicted)
    valid_actual = actual[valid_indices]
    valid_predicted = predicted[valid_indices]
    
    # Calculate RMSE
    if len(valid_actual) > 0:
        rmse = np.sqrt(mean_squared_error(valid_actual, valid_predicted))
        rmse_list.append(rmse)

# Calculate the average RMSE across all cities
average_rmse = np.mean(rmse_list)

print(f"RMSE for each city: {rmse_list}")
print(f"Average RMSE across all cities: {average_rmse}")


c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the 

Predictions for the first week of 2018 using SARIMA (Average Temperature):
city_id          C001       C002       C003      C004       C005       C007  \
2018-01-01   9.901990  14.770643  25.180545 -2.165690  27.431164  19.286122   
2018-01-02   9.847044  15.218716  25.403912 -1.268135  27.324541  18.176143   
2018-01-03  10.123795  15.306286  25.179772 -1.363100  27.426432  17.941259   
2018-01-04   9.813415  15.270328  25.294084 -1.506219  27.583221  17.931398   
2018-01-05   9.935299  15.242893  25.236624 -1.678773  27.601321  18.173331   
2018-01-06   9.919753  15.164702  25.125252 -1.862746  27.486208  18.430168   
2018-01-07   9.978284  15.354043  25.333412 -1.594865  27.468519  18.345155   

city_id         C008       C009       C010      C011  ...      C103  \
2018-01-01  8.962168  19.234629  20.078174 -2.101321  ...  2.356053   
2018-01-02  7.316179  19.350761  19.980468 -4.030275  ...  3.154017   
2018-01-03  6.434665  19.267395  20.001362 -4.974001  ...  3.257334   
2018-01-

c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\kauad\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
